# Preface
> I am equally taken a back by the popularity of the Wordle game as much as you guys are. I have wrote a script which provides you the word with most popular letters and is closest(statistically) to the best guess. Have doubts? Check yourself! 
> Here is the link to the [Wordle website](https://www.nytimes.com/games/wordle/index.html). The code has been provided with instructions(scroll to the end ⬇) to use it.

### 1. Loading and processing dataset
> I found this dataset from the creator of the **best mathematics channel on YouTube**, Grant Sanderson. His video on [3Blue1Brown](https://youtu.be/v68zYyaEmEA) inspired this idea. Thanks, Grant!!!

In [15]:
with open("../input/five-letter-words/word_list.txt") as data:
    word_list = [word.rstrip() for word in data]

print('Total number of five-letter words: {}\nSample:\n{}'.format(len(word_list), word_list[:10]))

### 2. Finding popular letters:
> I calculated frequency of each letter and created a dicionary object consisting all the **alphabets as keys** and its frequency as their values.

In [16]:
def frequency_dict(word_data):
    freq = {}
    for alphabet in list('abcdefghijklmnopqrstuvwxyz'):
        freq[alphabet] = 0
    for one in word_data:
        letters = list(one)
        for i in letters:
            freq[i] += 1
    freq_list = sorted(((value, key) for key, value in freq.items()), reverse=True)
    freq = dict((k,v) for v,k in freq_list)
    return freq

popular_letters = frequency_dict(word_list)
print(popular_letters)

### 3. Suggesting popular words:
> I sorted all the words based on popularity of its letters(basically its frequency in the dataset). This will help us in choosing the word that will eleminate highest wrong words from the available words. Two of the best guesses as our first guess will be '**SOARE**' and '**AROSE**'.

In [17]:
def suggest_best_words(list_of_words, famous_letters):
    word_fame = {}
    for word in list_of_words:
        word_weight = 0
        letters = list(word)
        letters = list(set(letters))
        for i in letters:
            word_weight += famous_letters[i]
        word_fame[word] = word_weight
    fame_tuple = sorted(((value, key) for key, value in word_fame.items()), reverse=True)
    word_fame = dict((k,v) for v,k in fame_tuple)
    return word_fame

best_words = suggest_best_words(word_list, popular_letters)
best_words10 = dict(list(best_words.items())[0:10])
print(best_words10)

### 4. Filtering the useful words:
> This will filter out the words based on the hint provided after our first guess. We will communicate **hint**(color coding after inputing the word in Wordle) to this code using following notations:
> * Gray: 0
> * Yellow: 1
> * Green: 2

In [18]:
def filtration(word, hint, look_into):
    filtered_words = []
    word = list(str(word))
    hint = list(str(hint))
    bad_letters, useful_letters = [], {}
    position = -1
    for letter, weight in zip(word, hint):
        position += 1
        if weight == '0':
            bad_letters.append(letter)                                      # identifying grey letters from the guessed word
        elif weight == '2':
            useful_letters[str(position)] = letter                          #identifying green letters from the guessed word
    for cur_word in look_into:
        unwanted_word = False
        list_of_letters = list(cur_word)
        if len(set(bad_letters).intersection(list_of_letters)) == 0:        #eliminating words consisting gray letters
            position = -1
            for letter, weight in zip(word, hint):
                position += 1
                for loc, cur_word_letter in enumerate(list_of_letters):     #eliminating words having yellow letter at the same place
                    if (cur_word_letter == letter) and (loc == position):
                        unwanted_word = True
                        break
                if weight == '2':
                    if letter == list_of_letters[position]:
                        filtered_words.append(cur_word)
                elif weight == '1':
                    if letter in list_of_letters:
                        if letter == list_of_letters[position] and cur_word in filtered_words:
                            filtered_words.remove(cur_word)
                        elif letter != list_of_letters[position] and unwanted_word == False:
                            filtered_words.append(cur_word)
                    else:
                        if cur_word in filtered_words:
                            filtered_words.remove(cur_word)
                if len(useful_letters) > 0:
                    for k,v in useful_letters.items():                          #eliminating word not consisting green letters
                        if  list_of_letters[int(k)] != v and cur_word in filtered_words:
                            filtered_words.remove(cur_word)

    print('Total Useful Words:', len(filtered_words))
    famous_words = suggest_best_words(filtered_words, popular_letters)
    return famous_words

### 5. How to use the code? Follow these simple steps:
1. Open Wordle game in another tab and type '**AROSE**' as your first guess.
2. Input the **hint**(color coding after inputing the word in Wordle) in the form of **string object** in the cell below and run the code.
> Use following notations to input hint.
> > * Gray letter : '0'
> > * Yellow letter: '1'
> > * Green letter : '2'

3. Select the word from the result for next attempt. (**Remider**: Use your brain. Do not select first word blindly.)

In [19]:
useful1 = filtration('arose', '10012', word_list)
useful2 = filtration('slate', '20122', useful1)
# useful3 = filtration('monie', '22022', useful2)
# useful4 = filtration('match', '02222', useful3)
print(list(useful2.items())[:20])